In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
import random

random.seed(666)
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

### Задание
1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:  
    1. бустинг   
    2. логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)  

2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)  
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного).  
Нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [2]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [3]:
def feature_processing(cat_cols, cont_cols, need_std=False):
    final_transformers = list()

    for cat_col in cat_cols:
        cat_transformer = Pipeline([
                    ('selector', FeatureSelector(column=cat_col)),
                    ('ohe', OHEEncoder(key=cat_col))
                ])
        final_transformers.append((cat_col, cat_transformer))

    for cont_col in cont_cols:
        pipes = [('selector', NumberSelector(key=cont_col))]
        # Добавляем нормализацю при необходимости
        if need_std:
            pipes.append(('standard', StandardScaler()))
            
        cont_transformer = Pipeline(pipes)
        final_transformers.append((cont_col, cont_transformer))
    feats = FeatureUnion(final_transformers)
    return feats

In [4]:
def make_preds(train_x, train_y, test_x, model, features_pipe):
    pipeline = Pipeline([
        ('features',features_pipe),
        ('classifier', model)
    ])

    #обучим наш пайплайн
    pipeline.fit(train_x, train_y)
    #наши прогнозы для тестовой выборки
    preds = pipeline.predict_proba(test_x)[:, 1]
    return preds

In [5]:
def get_scores(p, test_y):
    precision, recall, thresholds = precision_recall_curve(test_y, p)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    tn, fp, fn, tp = confusion_matrix(y_test, p>thresholds[ix]).ravel()
    
    scores = {
        'Threshold': thresholds[ix],
        'F-Score': fscore[ix],
        'Precision': precision[ix],
        'Recall': recall[ix],
        'true_positive': tp,
        'false_positive': fp,
        'true_negative': tn,
        'false_negative': fn
    }

    return scores 

In [6]:
df = pd.read_csv("./input/churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [7]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [8]:
feats = feature_processing(categorical_columns, continuous_columns)

In [9]:
lg_feats = feature_processing(categorical_columns, continuous_columns, need_std=True)

In [10]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [11]:
# %%time
# params_est = {
#     'loss': 'exponential',
#     'max_features': 14,
#     'random_state': 6
# }

# pipeline = Pipeline([
#     ('features',feats),
#     ('classifier', GradientBoostingClassifier(**params_est)),
# ])

# params={'classifier__n_estimators':[300, 10,  700],
#         'classifier__learning_rate':[0.01, 0.09, 0.7],
#         'classifier__subsample':[0.1, 1],
#         'classifier__min_samples_leaf':[10, 100, 500]
#         }

# grid = GridSearchCV(pipeline,
#                     param_grid=params,
#                     cv=6,
#                     refit=False)

# search = grid.fit(X_train, y_train)
# search.best_params_

# Wall time: 5min 50s
# {'classifier__learning_rate': 0.09,
#  'classifier__min_samples_leaf': 10,
#  'classifier__n_estimators': 300,
#  'classifier__subsample': 1}

In [12]:
params_est = {
    'n_estimators': 300,
    'min_samples_leaf': 10,
    'learning_rate': 0.09,
    'loss': 'exponential',
    'max_features': 14,
    'random_state': 6
}
gbc = GradientBoostingClassifier(**params_est)


In [13]:
gbc_preds = make_preds(X_train, y_train, X_test, gbc, feats)

In [14]:
gbc_scores = get_scores(gbc_preds, y_test)
print (f"Best Threshold={gbc_scores['Threshold']:.3f}\n \
    F-Score={gbc_scores['F-Score']:.3f}\n \
    Precision={gbc_scores['Precision']:.3f}\n \
    Recall={gbc_scores['Recall']:.3f}")

Best Threshold=0.325
     F-Score=0.638
     Precision=0.618
     Recall=0.660


In [15]:
lg = LogisticRegression()

In [16]:
lg_preds = make_preds(X_train, y_train, X_test, lg, lg_feats)

In [17]:
lg_scores = get_scores(lg_preds, y_test)
print (f"Best Threshold={lg_scores['Threshold']:.3f}\n \
    F-Score={lg_scores['F-Score']:.3f}\n \
    Precision={lg_scores['Precision']:.3f}\n \
    Recall={lg_scores['Recall']:.3f}")

Best Threshold=0.290
     F-Score=0.510
     Precision=0.462
     Recall=0.568


Все метрики говорят в пользу GradientBoosting, но я думаю что в задаче оттока наиболее важной метрикой будет Recall, т.к. необходимо минимизировать ошибки второго рода и найти всех клиентов, которых надо удержать.
Нельзя сказать что бы Precision, не играл важной роли, так как с низким Precision мы имеем риск трат на клиентов которые не собирались уходить.

In [18]:
print (f"\
     true_positive={gbc_scores['true_positive']}\n \
    false_positive={gbc_scores['false_positive']}\n \
    true_negative={gbc_scores['true_negative']}\n \
    false_negative={gbc_scores['false_negative']}")

     true_positive=335
     false_positive=208
     true_negative=1783
     false_negative=174


In [19]:
# расчитываем затраты на удержание, как цену удержания всех клиенттов которых модель посчитала уходящими
retention_costs = (gbc_scores['true_positive'] + gbc_scores['false_positive']) * 1.0

# расчитываем Ожидаемую прибыль, в расчете на каждого вовремя удержанного клиента
expected_profit = gbc_scores['true_positive'] * 2.0
# расчитываем общую эффективноть
economic_efficiency = expected_profit - retention_costs

if economic_efficiency > 0:
    print(f'Модель эффективна, экономическая эфективность составила {economic_efficiency}')
else:
    print(f'Модель не эффективна, экономическая эфективность составила {economic_efficiency}')

Модель эффективна, экономическая эфективность составила 127.0
